## __Vector Error-Correction Model__

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [20]:
inFile = "./Data/gasoline-combined.csv"
raw = pd.read_csv(inFile, parse_dates=True, index_col=0)

In [21]:
raw.head()

,Futures,Spot,Rate
Date,,,
1996-12-31,0.707,0.688,5.50000
1996-12-30,0.696,0.677,5.50391
1996-12-27,0.697,0.681,5.68750
1996-12-26,0.695,0.679,.
1996-12-24,0.696,0.677,5.66406


In [22]:
raw.tail()

,Futures,Spot,Rate
Date,,,
1986-06-06,0.410,0.421,7.18750
1986-06-05,0.414,0.431,7.18750
1986-06-04,0.412,0.418,7.18750
1986-06-03,0.442,0.436,7.18750
1986-06-02,0.449,0.468,7.12500


In [23]:
lnFut = raw.Futures.apply(np.log).dropna().values
lnSpot = raw.Spot.apply(np.log).dropna().values

In [24]:
df = pd.DataFrame({"lnS": lnSpot, "lnF": lnFut})

In [25]:
df.head()

,lnS,lnF
0,-0.373966,-0.346725
1,-0.390084,-0.362406
2,-0.384193,-0.360970
3,-0.387134,-0.363843
4,-0.390084,-0.362406


In [26]:
df.tail()

,lnS,lnF
2649,-0.865122,-0.891598
2650,-0.841647,-0.881889
2651,-0.872274,-0.886732
2652,-0.830113,-0.816445
2653,-0.759287,-0.800732


In [28]:
y = lnSpot
x = sm.add_constant(lnFut, prepend=True)

In [30]:
y[:10]

array([-0.37396644, -0.39008401, -0.38419297, -0.38713415, -0.39008401,
       -0.39749694, -0.37833644, -0.36672528, -0.36672528, -0.38713415])

In [31]:
x.shape

(2654, 2)

In [32]:
x[:5,:]

array([[ 1.        , -0.34672461],
       [ 1.        , -0.36240562],
       [ 1.        , -0.36096987],
       [ 1.        , -0.36384343],
       [ 1.        , -0.36240562]])

In [33]:
model = sm.OLS(y,x)

In [34]:
result = model.fit()

In [43]:
z = result.resid

In [50]:
DlnSpot = np.diff(lnSpot)

In [51]:
DlnSpot.shape

(2653,)

In [52]:
DlnFut = np.diff(lnFut)

In [53]:
DlnFut.shape

(2653,)

In [54]:
z.shape

(2654,)

In [68]:
n = DlnSpot.shape[0]
DlnSpot = DlnSpot.reshape((n,1))

In [70]:
DlnSpot.shape

(2653, 1)

In [71]:
n = DlnFut.shape[0]
DlnFut = DlnFut.reshape((n,1))

In [72]:
z = z[1:]
n = z.shape[0]
z = z.reshape((n,1))

In [75]:
X = np.hstack((DlnSpot, DlnFut, z))

In [76]:
X.shape

(2653, 3)

In [86]:
class BivariateVECM(object):
    def __init__(self, y1, y2):
        self.__y1 = y1
        self.__y2 = y2
        self.__lnY1 = np.log(y1)
        self.__lnY2 = np.log(y2)
        self.__DlnSpot0 = None
        self.__DlnFut0 = None
        self.__params1 = None
        self.__params2 = None
        self.__resids1 = None
        self.__resids2 = None
        self.__z = None
    
    def fit(self):
        y = self.__lnY1
        x = sm.add_constant(self.__lnY2, prepend=True)
        model = sm.OLS(y,x)
        result = model.fit()
        self.__z = result.resid[0]
        z = result.resid[:-2]
        n = z.shape[0]
        z = z.reshape((n,1))
        
        
        DlnSpot = np.diff(self.__y1)[:-1]
        n = DlnSpot.shape[0]
        DlnSpot = DlnSpot.reshape((n,1))
        self.__DlnSpot0 = DlnSpot[0]
        
        DlnFut = np.diff(self.__y2)[:-1]
        n = DlnFut.shape[0]
        DlnFut = DlnFut.reshape((n,1))
        self.__DlnFut0 = DlnFut[0]
        
        X = np.hstack((DlnSpot, DlnFut, z))
        X = sm.add_constant(X, prepend=True)
        
        Y1 = np.diff(self.__y1)[1:]
        eqn1 = sm.OLS(Y1, X)
        res1 = eqn1.fit()
        self.__params1 = res1.params
        self.__resids1 = res1.resid
        
        Y2 = np.diff(self.__y2)[1:]
        eqn2 = sm.OLS(Y2, X)
        res2 = eqn2.fit()
        self.__params2 = res2.params
        self.__resids2 = res2.resid
        
    
    def simulate(self, nsteps):
        ## Draw residuals with bootstrap
        u1 = np.random.choice(self.__resids1, size=nsteps, replace=True)
        u2 = np.random.choice(self.__resids2, size=nsteps, replace=True)
        z1 = np.random.choice(self.__z, size=nsteps, replace=True)
        y1 = np.empty(nsteps)
        y2 = np.empty(nsteps)
        
        y1[0] = self.__DlnSpot[0
        y2[0] = self.__DlnFut0
        
        for t in range(1, nsteps):
            pass
        

In [87]:
vecm = BivariateVECM(raw.Spot.values, raw.Futures.values)

In [88]:
vecm.fit()